### An example of using Oceananigans solvers
This example looks at two elliptic problem solvers that can be used for problems of the general form 

$  \nabla^2 \phi + \alpha \phi = \gamma $

the example shows two solvers. A direct FFT based solver and an iterative conjugate gradient solver. Th FFT solver is a very fast technique for very regular cartesian grids, the conjugate gradient works for an arbitrary geometry. 

In [ ]:
using Pkg
Pkg.add("Oceananigans")
Pkg.add("JLD2")
Pkg.add("Statistics")
Pkg.add("DataDeps")
Pkg.add("CairoMakie")
Pkg.add("Plots")
Pkg.add("KernelAbstractions")

In [ ]:
# Setup
using Oceananigans
using Oceananigans.Solvers
using Oceananigans.Models.NonhydrostaticModels: solve_for_pressure!
using Oceananigans.BoundaryConditions: fill_halo_regions!
using Plots
using KernelAbstractions: @kernel, @index, Event
using Oceananigans.Architectures: child_architecture, device, array_type
using Oceananigans.Utils: launch!
using Oceananigans.Operators: ∇²ᶜᶜᶜ, divᶜᶜᶜ
arch=CPU()
FT=Float64;

#### Add a few interfaces to operators for useful functions.
These illustrate the use of a couple of base operators, ∇²ᶜᶜᶜ and divᶜᶜᶜ . 

The  [`∇²ᶜᶜᶜ`](https://github.com/CliMA/Oceananigans.jl/blob/374e44f10ea6896a53b3258e59db4905e6dca281/src/Operators/laplacian_operators.jl#L36) operator computes 

$  \nabla^2 \phi  $ 

for a cell centered field on the Oceananigans grid.

The [`divᶜᶜᶜ`](https://github.com/CliMA/Oceananigans.jl/blob/374e44f10ea6896a53b3258e59db4905e6dca281/src/Operators/divergence_operators.jl#L16) operator computes

$ \nabla \cdot \phi $

for a cell centered field on the Oceananigans grid.


In [ ]:
function compute_∇²!(∇²ϕ, ϕ, arch, grid)
    fill_halo_regions!(ϕ)
    child_arch = child_architecture(arch)
    event = launch!(child_arch, grid, :xyz, ∇²!, ∇²ϕ, grid, ϕ, dependencies=Event(device(child_arch)))
    wait(device(child_arch), event)
    fill_halo_regions!(∇²ϕ)
    return nothing
end
@kernel function ∇²!(∇²f, grid, f)
    i, j, k = @index(Global, NTuple)
    @inbounds ∇²f[i, j, k] = ∇²ᶜᶜᶜ(i, j, k, grid, f)
end
@kernel function divergence!(grid, u, v, w, div)
    i, j, k = @index(Global, NTuple)
    @inbounds div[i, j, k] = divᶜᶜᶜ(i, j, k, grid, u, v, w)
end

#### Create grid and fields on grid

In [ ]:
Lx=1
Ly=1
Nx=200
Ny=200
grid=RectilinearGrid(arch,FT,size=(Nx,Ny),extent=(Lx,Ly), topology=(Periodic, Periodic, Flat))
u=XFaceField(grid);set!(u,rand(Nx,Ny));fill_halo_regions!(u)
v=YFaceField(grid);set!(v,rand(Nx,Ny));fill_halo_regions!(v)
w=ZFaceField(grid);set!(w,rand(Nx,Ny));fill_halo_regions!(w)
U=(u=u,v=v,w=w)
ϕ=CenterField(grid)
∇²ϕ=CenterField(grid)

#### Instance of solver (FFT based)
Create and then apply

In [ ]:
solverFFT = FFTBasedPoissonSolver(grid)

In [ ]:
solve_for_pressure!(ϕ.data, solverFFT, 1, U)

#### Check solution

In [ ]:
compute_∇²!(∇²ϕ, ϕ, arch, grid)

In [ ]:
divU=zeros(Nx, Ny, 1) |> array_type(arch)
event = launch!(arch, grid, :xyz, divergence!, grid, u.data, v.data, w.data, divU,
                    dependencies=Event(device(arch)))
wait(device(arch), event)

In [ ]:
maximum(divU[:,:,1].-interior(∇²ϕ)[:,:,1]), minimum(divU[:,:,1].-interior(∇²ϕ)[:,:,1])